In [170]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE


In [171]:
df=pd.read_csv("auto insurance.csv")

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_number                1000 non-null   int64  
 3   policy_bind_date             1000 non-null   object 
 4   policy_state                 1000 non-null   object 
 5   policy_csl                   1000 non-null   object 
 6   policy_deductable            1000 non-null   int64  
 7   policy_annual_premium        1000 non-null   float64
 8   umbrella_limit               1000 non-null   int64  
 9   insured_zip                  1000 non-null   int64  
 10  insured_sex                  1000 non-null   object 
 11  insured_education_level      1000 non-null   object 
 12  insured_occupation           1000 non-null   object 
 13  insured_hobbies    

In [173]:
df.isnull().sum().sort_values(ascending=False)

_c39                           1000
age                               0
incident_state                    0
incident_city                     0
incident_location                 0
incident_hour_of_the_day          0
number_of_vehicles_involved       0
property_damage                   0
bodily_injuries                   0
witnesses                         0
police_report_available           0
total_claim_amount                0
injury_claim                      0
property_claim                    0
vehicle_claim                     0
auto_make                         0
auto_model                        0
auto_year                         0
fraud_reported                    0
authorities_contacted             0
months_as_customer                0
collision_type                    0
insured_zip                       0
policy_number                     0
policy_bind_date                  0
policy_state                      0
policy_csl                        0
policy_deductable           

In [174]:
categorical_cols = df.select_dtypes(include=['object', 'category'])
categorical_cols.sample(5)

,policy_bind_date,policy_state,policy_csl,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,incident_date,incident_type,...,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,property_damage,police_report_available,auto_make,auto_model,fraud_reported
367,31-07-1999,OH,500/1000,FEMALE,Associate,adm-clerical,hiking,own-child,31-01-2015,Single Vehicle Collision,...,Minor Damage,Ambulance,VA,Northbend,2777 Solo Drive,NO,YES,Toyota,Highlander,N
166,22-07-2002,OH,250/500,FEMALE,Associate,craft-repair,board-games,unmarried,14-02-2015,Single Vehicle Collision,...,Major Damage,Fire,WV,Riverwood,8991 Embaracadero Ave,?,YES,Audi,A5,Y
540,09-01-2002,OH,500/1000,FEMALE,PhD,other-service,reading,wife,06-01-2015,Single Vehicle Collision,...,Major Damage,Other,VA,Hillsdale,9633 Rock Hwy,?,NO,Volkswagen,Jetta,N
632,05-07-2007,OH,500/1000,MALE,Masters,transport-moving,golf,not-in-family,10-01-2015,Single Vehicle Collision,...,Minor Damage,Fire,VA,Springfield,3808 5th Ave,NO,NO,Accura,MDX,N
915,12-12-1991,OH,250/500,FEMALE,High School,sales,chess,wife,13-02-2015,Single Vehicle Collision,...,Major Damage,Fire,SC,Hillsdale,4291 Sky Hwy,YES,?,Mercedes,C300,Y


In [175]:
categorical_cols.columns

Index(['policy_bind_date', 'policy_state', 'policy_csl', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'incident_date', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_state', 'incident_city', 'incident_location',
       'property_damage', 'police_report_available', 'auto_make', 'auto_model',
       'fraud_reported'],
      dtype='object')

In [176]:
categorical_cols['property_damage'].unique()

array(['YES', '?', 'NO'], dtype=object)

In [177]:
categorical_cols['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])  # ensure it's in datetime format
categorical_cols['policy_year'] = categorical_cols['policy_bind_date'].dt.year

categorical_cols['incident_date'] = pd.to_datetime(df['incident_date'])  # ensure it's in datetime format
categorical_cols['incident_year'] = categorical_cols['incident_date'].dt.year

categorical_cols['year_diff'] = categorical_cols['incident_year']-categorical_cols['policy_year']



C:\Users\brago\AppData\Local\Temp\ipykernel_17272\895121403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_cols['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])  # ensure it's in datetime format
C:\Users\brago\AppData\Local\Temp\ipykernel_17272\895121403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_cols['policy_year'] = categorical_cols['policy_bind_date'].dt.year
C:\Users\brago\AppData\Local\Temp\ipykernel_17272\895121403.py:4: SettingWithCopyWarning: 
A valu

In [178]:
categorical_cols=categorical_cols.drop(["insured_hobbies","incident_date","policy_bind_date","incident_location"],axis=1)
categorical_cols.shape

(1000, 20)

In [179]:
categorical_cols["incident_severity"].value_counts()

Minor Damage      354
Total Loss        280
Major Damage      276
Trivial Damage     90
Name: incident_severity, dtype: int64

In [180]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder,TargetEncoder

In [181]:
le=LabelEncoder()
ohe = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
ord_en=OrdinalEncoder()
encoder=ce.CountEncoder(normalize=True)
te=TargetEncoder()

In [182]:
# Column groups
freq_columns = ["policy_state", "policy_csl", "incident_type","insured_occupation",
                "insured_relationship", "authorities_contacted", "incident_state", "incident_city"]

ohe_columns = ["insured_sex", "property_damage", "police_report_available", "incident_type", "collision_type"]


te_columns = ["auto_make", "auto_model"]

In [183]:
# Label encode target
categorical_cols["fraud_reported"] = le.fit_transform(categorical_cols["fraud_reported"])

# Frequency Encoding
for col in freq_columns:
    categorical_cols[col] = encoder.fit_transform(categorical_cols[col])


# Target Encoding
for col in te_columns:
    categorical_cols[col] = te.fit_transform(categorical_cols[[col]], categorical_cols["fraud_reported"])


In [184]:
encoded_array = ohe.fit_transform(categorical_cols[ohe_columns])

encoded_cols = ohe.get_feature_names_out(ohe_columns)

# encoded_df = pd.DataFrame(encoded_array.toarray(), columns=encoded_cols, index=categorical_cols.index)
encoded_df = pd.DataFrame.sparse.from_spmatrix(encoded_array, columns=encoded_cols, index=categorical_cols.index)

categorical_cols = pd.concat([categorical_cols.drop(columns=ohe_columns), encoded_df], axis=1)
categorical_cols = categorical_cols.astype({col: int for col in categorical_cols.columns if categorical_cols[col].dtype == 'bool'})



In [185]:
categorical_cols

,policy_state,policy_csl,insured_education_level,insured_occupation,insured_relationship,incident_severity,authorities_contacted,incident_state,incident_city,auto_make,...,police_report_available_NO,police_report_available_YES,incident_type_0.084,incident_type_0.094,incident_type_0.403,incident_type_0.419,collision_type_?,collision_type_Front Collision,collision_type_Rear Collision,collision_type_Side Collision
0,0.352,0.351,MD,0.074,0.170,Major Damage,0.292,0.248,0.149,0.222572,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.310,0.351,MD,0.093,0.177,Minor Damage,0.292,0.110,0.134,0.349503,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.352,0.349,PhD,0.076,0.183,Minor Damage,0.292,0.262,0.149,0.272305,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.338,0.351,PhD,0.069,0.141,Major Damage,0.292,0.023,0.152,0.268329,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.338,0.300,Associate,0.076,0.141,Minor Damage,0.091,0.262,0.152,0.240144,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.352,0.300,Masters,0.074,0.141,Minor Damage,0.223,0.110,0.122,0.281765,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
996,0.338,0.349,PhD,0.085,0.155,Major Damage,0.223,0.248,0.145,0.235498,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
997,0.352,0.351,Masters,0.069,0.177,Minor Damage,0.292,0.110,0.152,0.245907,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
998,0.338,0.300,Associate,0.054,0.155,Major Damage,0.198,0.262,0.152,0.290052,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [186]:
categorical_cols["incident_severity"].unique()

array(['Major Damage', 'Minor Damage', 'Total Loss', 'Trivial Damage'],
      dtype=object)

In [187]:
from sklearn.preprocessing import OrdinalEncoder

# Initialize encoders with correct nesting
education_encoder = OrdinalEncoder(categories=[['JD', 'High School', 'Associate', 'MD', 'Masters', 'PhD', 'College']])
severity_encoder = OrdinalEncoder(categories=[['Trivial Damage', 'Minor Damage', 'Major Damage', 'Total Loss']])

# Apply encoding (reshape to 2D because sklearn expects 2D input)
categorical_cols['insured_education_level'] = education_encoder.fit_transform(categorical_cols[['insured_education_level']])

categorical_cols['incident_severity'] = severity_encoder.fit_transform(categorical_cols[['incident_severity']])

In [188]:
categorical_cols

,policy_state,policy_csl,insured_education_level,insured_occupation,insured_relationship,incident_severity,authorities_contacted,incident_state,incident_city,auto_make,...,police_report_available_NO,police_report_available_YES,incident_type_0.084,incident_type_0.094,incident_type_0.403,incident_type_0.419,collision_type_?,collision_type_Front Collision,collision_type_Rear Collision,collision_type_Side Collision
0,0.352,0.351,3.0,0.074,0.170,2.0,0.292,0.248,0.149,0.222572,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.310,0.351,3.0,0.093,0.177,1.0,0.292,0.110,0.134,0.349503,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.352,0.349,5.0,0.076,0.183,1.0,0.292,0.262,0.149,0.272305,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.338,0.351,5.0,0.069,0.141,2.0,0.292,0.023,0.152,0.268329,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.338,0.300,2.0,0.076,0.141,1.0,0.091,0.262,0.152,0.240144,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.352,0.300,4.0,0.074,0.141,1.0,0.223,0.110,0.122,0.281765,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
996,0.338,0.349,5.0,0.085,0.155,2.0,0.223,0.248,0.145,0.235498,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
997,0.352,0.351,4.0,0.069,0.177,1.0,0.292,0.110,0.152,0.245907,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
998,0.338,0.300,2.0,0.054,0.155,2.0,0.198,0.262,0.152,0.290052,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [189]:
numerical_cols = df.select_dtypes(include=['number'])
numerical_cols.sample(5)

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
962,150,37,649800,1000,1007.00,0,466238,30400,-89400,9,1,0,2,80080,12320,12320,55440,2013,NaN
587,463,59,638155,1000,979.73,0,601848,51700,0,14,2,1,2,72400,7240,14480,50680,1999,NaN
536,254,46,871432,2000,1472.43,0,619794,0,-90600,3,3,1,3,63580,5780,11560,46240,2004,NaN
123,22,25,824540,2000,1053.24,0,603527,51100,0,0,4,1,0,77130,8570,17140,51420,1995,NaN
495,261,46,156636,1000,870.55,0,465631,0,-49400,19,1,0,3,80280,13380,13380,53520,2013,NaN


In [190]:
numerical_cols=numerical_cols.drop(["policy_number","_c39"],axis=1)

In [191]:
numerical_cols.isnull().sum().sort_values(ascending=False)

months_as_customer             0
number_of_vehicles_involved    0
vehicle_claim                  0
property_claim                 0
injury_claim                   0
total_claim_amount             0
witnesses                      0
bodily_injuries                0
incident_hour_of_the_day       0
age                            0
capital-loss                   0
capital-gains                  0
insured_zip                    0
umbrella_limit                 0
policy_annual_premium          0
policy_deductable              0
auto_year                      0
dtype: int64

In [192]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [193]:
std_sc=StandardScaler()
sc_list=["policy_deductable","policy_annual_premium","total_claim_amount","injury_claim","property_claim","vehicle_claim","umbrella_limit",
         "capital-gains","capital-loss"]

for col in sc_list:
    numerical_cols[col]=std_sc.fit_transform(numerical_cols[[col]])

In [194]:
numerical_cols

,months_as_customer,age,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year
0,328,48,-0.222383,0.616705,-0.479476,466132,1.011331,0.953851,5,1,1,2,0.714257,-0.189283,1.165505,0.749653,2004
1,228,42,1.412784,-0.242521,1.697980,468176,-0.901927,0.953851,8,1,0,0,-1.807312,-1.363822,-1.372696,-1.823346,2007
2,134,29,1.412784,0.642233,1.697980,430632,0.358023,0.953851,7,3,2,3,-0.686362,0.054644,-0.736072,-0.785565,2007
3,256,41,1.412784,0.652886,2.133471,608117,0.853388,-1.267577,5,1,1,2,0.403135,-0.224130,-0.219722,0.677607,2014
4,228,44,-0.222383,1.341980,2.133471,610706,1.467210,-0.683741,20,1,0,1,-1.753121,-1.257232,-1.399654,-1.768252,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,-0.222383,0.222884,-0.479476,431289,-0.901927,0.953851,20,1,0,1,1.305049,2.051155,0.273817,1.224309,2006
996,285,41,-0.222383,0.739141,-0.479476,608177,1.643100,0.953851,23,1,2,3,2.111466,2.182342,2.214794,1.821868,2015
997,130,34,-1.039966,0.520739,0.826998,442797,0.358023,0.953851,4,3,2,3,0.558507,0.013648,0.020826,0.771902,1996
998,458,62,1.412784,0.411866,1.697980,441714,-0.901927,0.953851,2,1,0,1,-0.219110,-0.453708,-0.451976,-0.073580,1998


In [195]:
categorical_cols["fraud_reported"].value_counts()

0    753
1    247
Name: fraud_reported, dtype: int64

In [196]:
numerical_cols["insured_zip"] = te.fit_transform(numerical_cols[["insured_zip"]], categorical_cols["fraud_reported"])

In [221]:
skew_list=[]
for col in numerical_cols.columns:
    skew_list.append([numerical_cols[col].skew(),col])

In [222]:
skew_list

[[0.3621768477780205, 'months_as_customer'],
 [0.47898804709224163, 'age'],
 [0.4778872137362791, 'policy_deductable'],
 [0.004401994526608293, 'policy_annual_premium'],
 [1.8067121987145034, 'umbrella_limit'],
 [11.619650029525369, 'insured_zip'],
 [0.4788502295807904, 'capital-gains'],
 [-0.39147194299389365, 'capital-loss'],
 [-0.03558446643823673, 'incident_hour_of_the_day'],
 [0.502664087487584, 'number_of_vehicles_involved'],
 [0.014777103511243209, 'bodily_injuries'],
 [0.019636419856529662, 'witnesses'],
 [-0.5945819885102338, 'total_claim_amount'],
 [0.2648108784718182, 'injury_claim'],
 [0.37816877643082875, 'property_claim'],
 [-0.6210979312228692, 'vehicle_claim'],
 [-0.04828880711387396, 'auto_year']]

In [224]:
from sklearn.preprocessing import PowerTransformer

# Assuming 'numerical_cols' is a DataFrame of numerical features
pt = PowerTransformer(method='yeo-johnson', standardize=True)
numerical_cols["umbrella_limit"] = pt.fit_transform(numerical_cols[["umbrella_limit"]])


In [225]:
numerical_cols.shape,categorical_cols.shape

((1000, 17), (1000, 31))

In [226]:
merged_df=pd.concat([categorical_cols,numerical_cols],axis=1)
merged_df.shape

(1000, 48)

In [227]:
merged_df

,policy_state,policy_csl,insured_education_level,insured_occupation,insured_relationship,incident_severity,authorities_contacted,incident_state,incident_city,auto_make,...,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year
0,0.352,0.351,3.0,0.074,0.170,2.0,0.292,0.248,0.149,0.222572,...,0.953851,5,1,1,2,0.714257,-0.189283,1.165505,0.749653,2004
1,0.310,0.351,3.0,0.093,0.177,1.0,0.292,0.110,0.134,0.349503,...,0.953851,8,1,0,0,-1.807312,-1.363822,-1.372696,-1.823346,2007
2,0.352,0.349,5.0,0.076,0.183,1.0,0.292,0.262,0.149,0.272305,...,0.953851,7,3,2,3,-0.686362,0.054644,-0.736072,-0.785565,2007
3,0.338,0.351,5.0,0.069,0.141,2.0,0.292,0.023,0.152,0.268329,...,-1.267577,5,1,1,2,0.403135,-0.224130,-0.219722,0.677607,2014
4,0.338,0.300,2.0,0.076,0.141,1.0,0.091,0.262,0.152,0.240144,...,-0.683741,20,1,0,1,-1.753121,-1.257232,-1.399654,-1.768252,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.352,0.300,4.0,0.074,0.141,1.0,0.223,0.110,0.122,0.281765,...,0.953851,20,1,0,1,1.305049,2.051155,0.273817,1.224309,2006
996,0.338,0.349,5.0,0.085,0.155,2.0,0.223,0.248,0.145,0.235498,...,0.953851,23,1,2,3,2.111466,2.182342,2.214794,1.821868,2015
997,0.352,0.351,4.0,0.069,0.177,1.0,0.292,0.110,0.152,0.245907,...,0.953851,4,3,2,3,0.558507,0.013648,0.020826,0.771902,1996
998,0.338,0.300,2.0,0.054,0.155,2.0,0.198,0.262,0.152,0.290052,...,0.953851,2,1,0,1,-0.219110,-0.453708,-0.451976,-0.073580,1998


In [228]:
x=merged_df.drop(["fraud_reported"],axis=1)
y=merged_df["fraud_reported"]

In [229]:
from imblearn.over_sampling import SMOTE

smote=SMOTE(sampling_strategy='minority') 
x,y=smote.fit_resample(x,y)
y.value_counts()

d:\anaconda\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


1    753
0    753
Name: fraud_reported, dtype: int64

In [230]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score

In [231]:
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=42,stratify=y)


In [208]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
pred=lr.predict(X_test)
print("ROC_AUC Score : "  ,roc_auc_score(pred,y_test))
print(classification_report(pred,y_test))


ROC_AUC Score :  0.650298606502986
              precision    recall  f1-score   support

           0       0.60      0.66      0.63       137
           1       0.70      0.64      0.66       165

    accuracy                           0.65       302
   macro avg       0.65      0.65      0.65       302
weighted avg       0.65      0.65      0.65       302



d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
d:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [209]:
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)
pred=rfc.predict(X_test)
print("ROC_AUC Score : "  ,roc_auc_score(pred,y_test))
print(classification_report(pred,y_test))

d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


ROC_AUC Score :  0.8757590977580934
              precision    recall  f1-score   support

           0       0.96      0.80      0.87       181
           1       0.76      0.95      0.85       121

    accuracy                           0.86       302
   macro avg       0.86      0.88      0.86       302
weighted avg       0.88      0.86      0.86       302



d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [210]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train,y_train)
pred=dtc.predict(X_test)
print("ROC_AUC Score : "  ,roc_auc_score(pred,y_test))
print(classification_report(pred,y_test))

ROC_AUC Score :  0.8018010103228641
              precision    recall  f1-score   support

           0       0.82      0.79      0.81       157
           1       0.78      0.81      0.80       145

    accuracy                           0.80       302
   macro avg       0.80      0.80      0.80       302
weighted avg       0.80      0.80      0.80       302



d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [211]:
svc=SVC()
svc.fit(X_train,y_train)
pred=svc.predict(X_test)
print("ROC_AUC Score : "  ,roc_auc_score(pred,y_test))
print(classification_report(pred,y_test))

d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


ROC_AUC Score :  0.5070958646616541
              precision    recall  f1-score   support

           0       0.64      0.51      0.56       190
           1       0.38      0.51      0.43       112

    accuracy                           0.51       302
   macro avg       0.51      0.51      0.50       302
weighted avg       0.54      0.51      0.51       302



In [232]:
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

In [233]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rfc,
    param_distributions=param_dist,
    n_iter=50,                # number of combinations to try
    cv=5,                     # 5-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1                 # use all processors
)

In [234]:
random_search.fit(X_train, y_train)
pred=random_search.predict(X_test)
print("ROC_AUC Score : "  ,roc_auc_score(pred,y_test))
print(classification_report(pred,y_test))

Fitting 5 folds for each of 50 candidates, totalling 250 fits


d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


ROC_AUC Score :  0.8808535333244053
              precision    recall  f1-score   support

           0       0.94      0.83      0.88       171
           1       0.81      0.93      0.87       131

    accuracy                           0.87       302
   macro avg       0.87      0.88      0.87       302
weighted avg       0.88      0.87      0.87       302



d:\anaconda\lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [236]:
random_search.best_params_

{'n_estimators': 300,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': True}